<a href="https://colab.research.google.com/github/almedida/thesis/blob/main/simulation_new_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Install Packages**##

In [210]:
rm(list = ls())

In [3]:
install.packages('pacman')
library(pacman, devtools)
p_load("tidyverse", "matrixTests", "gtools", "tmvtnorm")

#install BiocManager
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

#install limma and qvalue packages
BiocManager::install(c("limma", "qvalue"))
library(limma)
library(qvalue)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.2 (2021-11-01)

Warning message:
“package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'limma'”
Installing package(s) 'qvalue'

also installing the dependencies ‘plyr’, ‘reshape2’


Old packages: 'cli', 'clipr', 'colorspace', 'crayon', 'evaluate', 'glue',
  'jsonlite', 'openssl', 'rmarkdown', 'tidyselect', 'tinytex', 'xfun', 'yaml',
  'mgcv', 'survival'



##***Load Dataset***

In [211]:
  genej_sim_dataj = read.csv("genej_sim_data2.csv", header = TRUE, sep = ",")

##Simulation function##

In [212]:
# simulation = function(){
  
  m = 10000 #no of genes to be selected from the dataset
  n_samples = 4 #no of samples
  n_mu = 2 #mu multipling delta
  n_de = 1000 #number of m11

  # genej_sim_data = read.csv("genej_sim_data2.csv", header = TRUE, sep = ",")

  N = ncol(genej_sim_dataj) #N = number columns (samples in the dataset)
  n_gene = nrow(genej_sim_dataj) #n_gene = no of genes (rows in the dataset)
  
  randomly_selected_genes = sample(n_gene, m) #randomly selected genes of size m from the total genes

  #10000 selected data corresponding to the randomly selected genes (EE genes)
  randomly_selected_data = genej_sim_dataj[randomly_selected_genes, ]
  genej_sim_data = randomly_selected_data

  genej_std = apply(genej_sim_data, 1, sd)  #calculate std for all genes

 
  randomly_selected_samples  = sample(N, 2*n_samples) #randomly select samples of size 2*n_trt_grp 
                                                    #for treatment group
  treatment_group1 = randomly_selected_samples[1:n_samples]
  treatment_group2 = randomly_selected_samples[(n_samples + 1):(2*n_samples)]

  treatment_group1_data = genej_sim_data[treatment_group1]

  #group 2 data
  treatment_group2_data = genej_sim_data[treatment_group2]

  #generate treatment effects, j_effects=10000 for the m genes
  mu = n_mu*genej_std
  treatment_effect_j = rnorm(m, mu, genej_std)

  #add 10k treatments effects generated to group 2 data
  de_treatment_group2 = treatment_group2_data + treatment_effect_j

  n_treated_genes_grp2 = nrow(de_treatment_group2)

  #we extract m genes from DE_treatment_group2 (genes with treatment effects)
  de_genes_index = sample(n_treated_genes_grp2, n_de)

  #we copied untreated (genes without effects) genes from treatment_group2_data

  treated_genes_group2 = treatment_group2_data

  #we replaced the m untreated genes (without treatment effects) with the corresponding treated genes(with treatment effects)
  treated_genes_group2[de_genes_index, ] = de_treatment_group2[de_genes_index, ]

  group1_data = treatment_group1_data
  group2_data <- treated_genes_group2

  # group2_data = treated_genes_group2

  #student t-test
  ttest <- row_t_equalvar(group1_data, group2_data, alternative = "two.sided", mu = 0, conf.level = 0.95)

  pvalue <- ttest[13]

  adj_pvalue <- p.adjust(data.matrix(pvalue), "BH")
  adj_pvalue_df <-  data.frame(adj_pvalue)

  #limma 
  limma_gene_exp <- c(group1_data, group2_data)
  limma_gene_exp <- data.frame(limma_gene_exp)

  expr_list_10k <- factor(
  x = c(rep("group1", n_samples), rep("group2", n_samples)),
  levels=c("group1","group2")            # Set group 1 to be the first level
  )

  design <- model.matrix(~expr_list_10k)          # Remove the zero

  fit <- lmFit(limma_gene_exp, design)
  fit <- eBayes(fit)
  results <- decideTests(fit)

  limma_pvalue <- fit$p.value[, 2]
  limma_pvalue <- data.frame(limma_pvalue)

  adj_limma_pvalue <- data.matrix(limma_pvalue)
  adj_limma_pvalue <- p.adjust(adj_limma_pvalue, "BH")
  adj_limma_pvalue <- data.frame(adj_limma_pvalue)


  pval_raw <- c(pvalue, limma_pvalue)
  pval_raw <- data.frame(pval_raw)
  pvals1 <- (as.matrix(pval_raw[,1]))
  pvals2 <- (as.matrix(pval_raw[,2]))

  print(head(pval_raw))
  print(head(pvals1))
  print(head(pvals2))
  # pvalues <- cbind(c(pvals1), c(pvals2))
  # colnames(pvalues) = c("pvals1", "pvals2")
  # print(list(pval_raw, pvals1, pvals2))

# }

       pvalue limma_pvalue
1 0.224160944 0.1685704031
2 0.137986637 0.0871356348
3 0.253470269 0.3066384209
4 0.418950271 0.3750277983
5 0.001886567 0.0003709841
6 0.110503485 0.1787782559
            [,1]
[1,] 0.224160944
[2,] 0.137986637
[3,] 0.253470269
[4,] 0.418950271
[5,] 0.001886567
[6,] 0.110503485
             [,1]
[1,] 0.1685704031
[2,] 0.0871356348
[3,] 0.3066384209
[4,] 0.3750277983
[5,] 0.0003709841
[6,] 0.1787782559


## **Proposed Method**

**Calculate cut-off points using Histogram-based method**

calc.cutoff slightly modifies the estimate.m0 function to return the appropriate p-value cutoff where all p-values greater than the cutoff are assumed to come from null cases.

In [194]:
values.list <- simulation()
pval_raw <- values.list[1])
pvals1 <- values.list[2]
pvals2 <- values.list[3]

In [214]:

calc.cutoff = function(p, B = 20, max=1){


  m <- length(p)
  m0 <- m
  bin <- c(-0.1, (1:B)/B*max)
  bin.counts=rep(0,B)

  for(i in 1:B){
    bin.counts[i]=sum((p>bin[i])&(p<=bin[i+1]))
  }

  tail.means <- rev(cumsum(rev(bin.counts))/(1:B))
  temp <- bin.counts - tail.means
  index <- min((1:B)[temp <= 0])
  cutoff2 <- (index)/B*max
  if(cutoff2 == 1) {cutoff2 <- 1-1/B}

  return(cutoff2)

}

cutoff_value1 = calc.cutoff(pvals1, B=20, max=1)
cutoff_value2 = calc.cutoff(pvals2, B=20, max=1)

cutoff = cbind(c(cutoff_value1), c(cutoff_value2))

colnames(cutoff) = c("cutoff_value1", "cutoff_value2")

cutoff

cutoff_value1,cutoff_value2
0.95,0.75


**Filter dataset using cutoff points**

selecting pvalues in the upper-right quadrant using Histogram-Based method
defined as lambda1 = cutoffvalue1 and lambda2 = cutoffvalue2 for pvalue 1 and pvalue 2 respectively.
lambda1, lambda2 - probability of pvalue 1 and pvalue 2 in the upper-right quadrant respectively

In [215]:
p_vals = pval_raw  %>% filter(pvalue >=cutoff_value1, limma_pvalue>=cutoff_value2)

convert selected pvalues to z values

In [216]:
z_val = as.data.frame(qnorm(as.matrix(p_vals), lower.tail = TRUE))
colnames(z_val) = c("zvals1", "zvals2")

zvals1 <- (as.data.frame(as.matrix(z_val[,1])))
zvals2 <- (as.data.frame(as.matrix(z_val[,2])))

convert lambda(truncation points) to z values

In [217]:
z_val_extremums = as.data.frame(qnorm(as.matrix(cbind(c(cutoff_value1,1),c(cutoff_value2,1))), lower.tail = TRUE))

min_z1 <- z_val_extremums[1,1]
min_z2 <- z_val_extremums[1,2]

**Estimating m00s**

m0.1 - DE genes in experiment 1
m0.2 - DE genes in experiment 2
m11 - DE genes in both experiments
m00 - EE genes in both experiments

In [218]:
estimate.m0s <- function(p1, p2, B=20){

  

  m <- length(p1)

  ##find lambda cutoffs using histogram-based method
  c1 <- calc.cutoff(p1, B=B, max=1)
  c2 <- calc.cutoff(p2, B=B, max=1)

  ##estimate m0 for experiment 1
  ind1 <- (p1>=c1)
  m0.1 <- sum(ind1)/(1-c1)
  m0.1 <- min(m0.1, 10000)

  ##estimate m0 for experiment 2  
  ind2 <- (p2>=c2)
  m0.2 <- sum(ind2)/(1-c2)
  m0.2 <- min(m0.2, 10000)


  ##estimate m00
  ind12 <- ind1 & ind2
  nA <- sum(ind12)
  #pA <- (1-c1)*(1-c2)
  #m00 <- nA/pA
  
  #here, we used converted pvalues to z values to estimnate m00
  # density function for each row of the bivariate z values (x) and 
  # estimated parameters(rho)
  density = function(x, rho)
  {
    sigma = matrix(c(1, rho, rho, 1), 2, 2)
    z = dtmvnorm(x, mean = c(0,0), sigma = sigma, lower = c(min_z1, min_z2))
  }
  
  # log likelihood of the joint densities
  log_likelihood_fn = function(rho){
    
    joint_likelihood = z_val %>% split(.$zvals2) %>% map_dfr(~density(c(.$zvals1,.$zvals2),rho))    
    return(-sum(log(joint_likelihood)))
    
  }
  
  #MLE of the log likelihood function
  optimal_rho = optimize(log_likelihood_fn, lower = -1, upper = 1 )
  optimal_rho = as.data.frame(optimal_rho)
  
  #probability of a random variable greater than cutoff values 
  rho = as.numeric(optimal_rho[1])
  obj_value = optimal_rho[2]
  pA = pmvnorm(lower=c(min_z1, min_z2), upper=c(Inf, Inf), mean=c(0,0), sigma = matrix(c(1, rho, rho, 1), 2, 2))
  

  m00 <- nA/pA
  m00 <- min(m00, 10000)
  
  
  ##estimate m11
  m11 <- sum(m - m0.1 - m0.2 + m00)
  if (m0.1 == 10000 || m0.2 == 10000 || m00 == 10000){
    m11 = 0
  }
  
  ret <- list()
  ret$ms <- c(obj_value, rho, m, m0.1, m0.2, m00, m11)
  names(ret$ms) <- c("obj_value", "optimal_rho", "m", "m0.1", "m0.2", "m00", "m11")
  ret$cutoffs <- c(c1, c2)
  return(ret)
}

estimate.m0s(pvals1, pvals2, B=20)

$ms
$ms$obj_value
[1] -306.8124

$ms$optimal_rho
[1] 0.9981253

$ms$m
[1] 10000

$ms$m0.1
[1] 4780

$ms$m0.2
[1] 5160

$ms$m00
[1] 4780

$ms$m11
[1] 4840


$cutoffs
[1] 0.95 0.75

# **Orr, 2012 Method**

In [ ]:
estimate.m0s <- function(p1, p2, B=20){
  m <- length(p1)

  ##find lambda cutoffs using histogram-based method
  c1 <- calc.cutoff(p1, B=B, max=1)
  c2 <- calc.cutoff(p2, B=B, max=1)

  ##estimate m0 for experiment 1
  ind1 <- (p1>=c1)
  m0.1 <- sum(ind1)/(1-c1)
  m0.1 <- min(m0.1, 10000)
  

  ##estimate m0 for experiment 2  
  ind2 <- (p2>=c2)
  m0.2 <- sum(ind2)/(1-c2)
  m0.2 <- min(m0.2, 10000)

  ##estimate m00
  ind12 <- ind1 & ind2
  nA <- sum(ind12)
  pA <- (1-c1)*(1-c2)
  m00 <- nA/pA
  m00 <- min(m00, 10000)


  ##estimate m11
  m11 <- sum(m - m0.1 - m0.2 + m00)
  if (m00 == 10000 || m0.1 == 10000 || m0.2 == 10000){
    m11 = 0
  }
  
  ret <- list()
  ret$ms <- c(m, m0.1, m0.2, m11, m00)
  names(ret$ms) <- c("m", "m0.1", "m0.2", "m11","m00")
  ret$cutoffs <- c(c1, c2)
  return(ret)
}

estimate.m0s(pvals1, pvals2, B=20)

$ms
        m      m0.1      m0.2       m11       m00 
10000.000  9336.667 10000.000     0.000  9828.070 

$cutoffs
[1] 0.70 0.05

# **Intersection Method**

In [ ]:
adj_pvalue_df <- data.frame(adj_pvalue)
# head(adj_pvalue_df)

In [ ]:
pvalue_p <- adj_pvalue_df[rowSums(adj_pvalue_df[1]<=0.05), ]
pvalue_05 <- length(pvalue_p)
#pvalue_05

In [ ]:
limma_p <- adj_limma_pvalue[rowSums(adj_limma_pvalue[1]<=0.05), ]
limma_05 <- length(limma_p) 
limma_05

[1] 0

In [ ]:
limma_ttest <- c(adj_pvalue_df, adj_limma_pvalue)
limma_ttest <- data.frame(limma_ttest)

In [ ]:
limma_ttest_p <- limma_ttest[rowSums((limma_ttest[1]<=0.05) & (limma_ttest[2]<=0.05)), ]
limma_ttest_05 <- nrow(limma_ttest_p)
limma_ttest_05

[1] 0

In [ ]:
pvalue_p2 <- adj_pvalue_df[rowSums(adj_pvalue_df[1]<=0.1), ]
pvalue_1 <- length(pvalue_p2)
pvalue_1

[1] 0

In [ ]:
limma_p2 <- adj_limma_pvalue[rowSums(adj_limma_pvalue[1]<=0.1), ]
limma_1 <- length(limma_p2) 
limma_1

In [ ]:
# write.csv(limma_ttest, "adj_pvalue_limma.csv", row.names=FALSE)

In [ ]:
limma_ttest_p2 <- limma_ttest[rowSums((limma_ttest[1]<=0.1) & (limma_ttest[2]<=0.1)), ]
limma_ttest_1 <- nrow(limma_ttest_p2)
limma_ttest_1

[1] 0